<a href="https://colab.research.google.com/github/bsun1220/UCBTradingComp2022/blob/main/UCBerkeley2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def naiveEV(seen_set, a, b, players):
    unseen_set = set(range(a,b+1)) - seen_set
    unknown_card_ev = sum(unseen_set)/len(unseen_set)
    return len(seen_set) + unknown_card_ev*(players-len(seen_set))